In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import tensorflow as tf

In [2]:
df1 = pd.read_csv("train_MpHjUjU.csv")
test = pd.read_csv("test_hXY9mYw.csv")

In [4]:
df1['Target'] =np.where(df1['LastWorkingDate'].isnull(),0,1)
df1


,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target
0,01-01-2016,1,28,Male,C23,Master,57387,24-12-2015,NaN,1,1,2381060,2,0
1,01-02-2016,1,28,Male,C23,Master,57387,24-12-2015,NaN,1,1,-665480,2,0
2,01-03-2016,1,28,Male,C23,Master,57387,24-12-2015,11-03-2016,1,1,0,2,1
3,01-11-2017,2,31,Male,C7,Master,67016,06-11-2017,NaN,2,2,0,1,0
4,01-12-2017,2,31,Male,C7,Master,67016,06-11-2017,NaN,2,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,01-08-2017,2788,30,Male,C27,Master,70254,08-06-2017,NaN,2,2,740280,3,0
19100,01-09-2017,2788,30,Male,C27,Master,70254,08-06-2017,NaN,2,2,448370,3,0
19101,01-10-2017,2788,30,Male,C27,Master,70254,08-06-2017,NaN,2,2,0,2,0
19102,01-11-2017,2788,30,Male,C27,Master,70254,08-06-2017,NaN,2,2,200420,2,0


In [7]:
a1=df1.groupby('Emp_ID')['Emp_ID','Age','Gender','City','Education_Level','Salary','Joining Designation','Designation',
                       'Quarterly Rating','Target'].tail(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [8]:
a2 = df1.groupby('Emp_ID').agg({'Total Business Value':'sum'})

In [10]:
a2.reset_index(drop=True,inplace=True)

In [12]:
a1.reset_index(drop=True,inplace=True)

In [13]:
df2 = pd.concat([a1,a2],axis=1,ignore_index=False)

In [14]:
df2

,Emp_ID,Age,Gender,City,Education_Level,Salary,Joining Designation,Designation,Quarterly Rating,Target,Total Business Value
0,1,28,Male,C23,Master,57387,1,1,2,1,1715580
1,2,31,Male,C7,Master,67016,2,2,1,0,0
2,4,43,Male,C13,Master,65603,2,2,1,1,350000
3,5,29,Male,C9,College,46368,1,1,1,1,120360
4,6,31,Female,C11,Bachelor,78728,3,3,2,0,1265000
...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,34,Male,C24,College,82815,2,3,4,0,21748820
2377,2785,34,Female,C9,College,12105,1,1,1,1,0
2378,2786,45,Male,C19,College,35370,2,2,1,1,2815090
2379,2787,28,Female,C20,Master,69498,1,1,1,1,977830


In [15]:
df3 = df2.copy()

In [16]:
le = LabelEncoder()
var_mod = df3.select_dtypes(include='object').columns
for i in var_mod:
    df3[i] = le.fit_transform(df3[i].astype(str))

In [17]:
df3

,Emp_ID,Age,Gender,City,Education_Level,Salary,Joining Designation,Designation,Quarterly Rating,Target,Total Business Value
0,1,28,1,15,2,57387,1,1,2,1,1715580
1,2,31,1,26,2,67016,2,2,1,0,0
2,4,43,1,4,2,65603,2,2,1,1,350000
3,5,29,1,28,1,46368,1,1,1,1,120360
4,6,31,0,2,0,78728,3,3,2,0,1265000
...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,34,1,16,1,82815,2,3,4,0,21748820
2377,2785,34,0,28,1,12105,1,1,1,1,0
2378,2786,45,1,10,1,35370,2,2,1,1,2815090
2379,2787,28,0,12,2,69498,1,1,1,1,977830


In [19]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')

In [20]:
x=df3.drop('Target',axis=1)
y=df3['Target']

In [21]:
X_sm, y_sm = smote.fit_resample(x, y)

y_sm.value_counts()

1    1616
0    1616
Name: Target, dtype: int64

In [22]:
from  sklearn import preprocessing

In [23]:
scaled_inputs_train = preprocessing.scale(X_sm)

In [24]:
(scaled_inputs_train.shape[0])

3232

In [27]:
xtrain,xtest,ytrain,ytest=train_test_split(scaled_inputs_train,y_sm,test_size=0.25,random_state=121)

In [28]:
xtrain.shape[0]

2424

In [29]:
samples_count = xtrain.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.2 * samples_count)



# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = xtrain[:train_samples_count]
train_targets = ytrain[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = xtrain[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = ytrain[train_samples_count:train_samples_count+validation_samples_count]

In [57]:
xtrain.shape

(2424, 10)

**MODEL**

In [61]:
input_size = 10
output_size = 1
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    tf.keras.layers.Dense(output_size, activation= 'sigmoid') # output layer
])

In [62]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [89]:
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model

model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )

Epoch 1/100
20/20 - 0s - loss: 0.3374 - accuracy: 0.8437 - val_loss: 0.3483 - val_accuracy: 0.8388 - 230ms/epoch - 11ms/step
Epoch 2/100
20/20 - 0s - loss: 0.3326 - accuracy: 0.8556 - val_loss: 0.3566 - val_accuracy: 0.8306 - 82ms/epoch - 4ms/step
Epoch 3/100
20/20 - 0s - loss: 0.3309 - accuracy: 0.8608 - val_loss: 0.3582 - val_accuracy: 0.8202 - 71ms/epoch - 4ms/step


In [90]:
ypred=model.predict(xtest)

In [37]:
from sklearn.metrics import confusion_matrix , classification_report

In [67]:
type(ypred)

numpy.ndarray

In [91]:
ypred1 = pd.DataFrame(ypred,columns=['target'])

In [92]:
ypred1

,target
0,0.903193
1,0.322202
2,0.999862
3,0.052208
4,0.374942
...,...
803,0.016081
804,0.504312
805,0.873258
806,0.954523


In [93]:
ypred2 = ypred1['target'].map(lambda t: 0 if t<0.5 else 1)

In [94]:
ypred2 

0      1
1      0
2      1
3      0
4      0
      ..
803    0
804    1
805    1
806    1
807    0
Name: target, Length: 808, dtype: int64

In [95]:
print(classification_report(ytest,ypred2))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71       399
           1       0.71      0.80      0.75       409

    accuracy                           0.73       808
   macro avg       0.74      0.73      0.73       808
weighted avg       0.74      0.73      0.73       808



In [96]:
test1=pd.merge(test,df3,on='Emp_ID')

In [97]:
test1

,Emp_ID,Age,Gender,City,Education_Level,Salary,Joining Designation,Designation,Quarterly Rating,Target,Total Business Value
0,394,34,0,12,2,97722,2,4,3,0,27458220
1,173,39,1,20,1,56174,1,3,3,0,19866290
2,1090,39,1,4,1,96750,2,4,2,0,50382490
3,840,40,0,27,1,88813,1,4,2,0,17134040
4,308,32,1,24,2,188418,2,5,2,0,58024490
...,...,...,...,...,...,...,...,...,...,...,...
736,2134,40,1,21,1,116006,2,5,1,0,59696450
737,2255,40,1,17,1,133489,3,4,3,0,36398090
738,448,37,1,1,0,65389,2,4,3,0,30318780
739,1644,48,0,28,0,105513,2,4,1,0,28842760


In [98]:
test1=test1.drop('Target',axis=1)

In [99]:
fpredictions=model.predict(test1)

In [100]:
fpredictions

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],